### Author: Rodolfo Elenes

Date Created: 8/5/2025

Change log:
8/5/2025 - Initialized

# Notebook to do list
    1.) None
    
# Enhancements
    1.) Create function that goes through seasons before 2024 season

In [1]:
import pandas as pd
import datetime
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Get Runningbacks

In [2]:
# Setup the starting dataframe

players = pd.read_csv("../src/roster_2024.csv")
df = players[(players['depth_chart_position'] == 'RB') & (players['status'] == 'ACT')]
display(df)

,season,team,position,depth_chart_position,jersey_number,status,full_name,first_name,last_name,birth_date,...,game_type,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number
82,2024,PIT,RB,RB,84.0,ACT,Cordarrelle Patterson,Cordarrelle,Patterson,3/17/1991,...,WC,A01,Cordarrelle,PAT387075,39975,32005041-5438-7075-4e25-a1072ce2f5ac,2013,2013.0,MIN,29.0
151,2024,MIA,RB,RB,31.0,ACT,Raheem Mostert,Dominique,Mostert,4/9/1992,...,REG,A01,Raheem,MOS822492,42718,32004d4f-5382-2492-b0f0-aec08c3525cf,2015,2015.0,NaN,NaN
251,2024,BAL,RB,RB,22.0,ACT,Derrick Henry,Derrick,Henry,1/4/1994,...,DIV,A01,Derrick,HEN602843,43334,32004845-4e60-2843-9b64-3cd8fac319c3,2016,2016.0,TEN,45.0
323,2024,MIN,RB,RB,33.0,ACT,Aaron Jones,Aaron,Jones,12/2/1994,...,WC,A01,Aaron,JON026624,44995,32004a4f-4e02-6624-33bd-3ba151b7013d,2017,2017.0,GB,182.0
355,2024,KC,RB,RB,34.0,ACT,Samaje Perine,Samaje,Perine,9/16/1995,...,SB,A01,Samaje,PER095343,44927,32005045-5209-5343-a8b2-b2085b20ed79,2017,2017.0,WAS,114.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3022,2024,NE,RB,RB,26.0,ACT,Terrell Jennings,Terrell,Jennings,1/3/2001,...,REG,A01,Terrell,JEN627185,57841,32004a45-4e62-7185-d6ac-81311d406644,2024,2024.0,NaN,NaN
3059,2024,NYJ,RB,RB,0.0,ACT,Braelon Allen,Braelon,Allen,1/20/2004,...,REG,A01,Braelon,ALL063734,57255,3200414c-4c06-3734-6e11-4c57194f79b4,2024,2024.0,NYJ,134.0
3063,2024,NYJ,RB,RB,32.0,ACT,Isaiah Davis,Isaiah,Davis,2/21/2002,...,REG,A01,Isaiah,DAV410574,57294,32004441-5641-0574-531b-0bc55bd5bc41,2024,2024.0,NYJ,173.0
3135,2024,MIA,RB,RB,25.0,ACT,Jaylen Wright,Jaylen,Wright,4/1/2003,...,REG,A01,Jaylen,WRI398099,57241,32005752-4939-8099-f6ee-7772cc125a42,2024,2024.0,MIA,120.0


# Exploratory Data Work

In [3]:
display(df.head()) #visualize the table
print(df.info()) #check data structure
print("\n")
print(df.isnull().any()) #check for nulls

#check duplicate rows
if df.duplicated().any():
    print('\nDuplicates found.')
else:
    print('\nNo duplicates found.')

,season,team,position,depth_chart_position,jersey_number,status,full_name,first_name,last_name,birth_date,...,game_type,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number
82,2024,PIT,RB,RB,84.0,ACT,Cordarrelle Patterson,Cordarrelle,Patterson,3/17/1991,...,WC,A01,Cordarrelle,PAT387075,39975,32005041-5438-7075-4e25-a1072ce2f5ac,2013,2013.0,MIN,29.0
151,2024,MIA,RB,RB,31.0,ACT,Raheem Mostert,Dominique,Mostert,4/9/1992,...,REG,A01,Raheem,MOS822492,42718,32004d4f-5382-2492-b0f0-aec08c3525cf,2015,2015.0,NaN,NaN
251,2024,BAL,RB,RB,22.0,ACT,Derrick Henry,Derrick,Henry,1/4/1994,...,DIV,A01,Derrick,HEN602843,43334,32004845-4e60-2843-9b64-3cd8fac319c3,2016,2016.0,TEN,45.0
323,2024,MIN,RB,RB,33.0,ACT,Aaron Jones,Aaron,Jones,12/2/1994,...,WC,A01,Aaron,JON026624,44995,32004a4f-4e02-6624-33bd-3ba151b7013d,2017,2017.0,GB,182.0
355,2024,KC,RB,RB,34.0,ACT,Samaje Perine,Samaje,Perine,9/16/1995,...,SB,A01,Samaje,PER095343,44927,32005045-5209-5343-a8b2-b2085b20ed79,2017,2017.0,WAS,114.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 82 to 3136
Data columns (total 36 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   season                   100 non-null    int64  
 1   team                     100 non-null    object 
 2   position                 100 non-null    object 
 3   depth_chart_position     100 non-null    object 
 4   jersey_number            100 non-null    float64
 5   status                   100 non-null    object 
 6   full_name                100 non-null    object 
 7   first_name               100 non-null    object 
 8   last_name                100 non-null    object 
 9   birth_date               100 non-null    object 
 10  height                   100 non-null    float64
 11  weight                   100 non-null    int64  
 12  college                  100 non-null    object 
 13  gsis_id                  100 non-null    object 
 14  espn_id                 

In [4]:
# check experience in league splits

df_numeric = df.select_dtypes(include=['number'])
df_numeric = df_numeric['years_exp'].value_counts().reset_index()
df_numeric.columns = ['years_exp', 'count']
display(df_numeric.sort_values("years_exp"))

,years_exp,count
0,0,19
2,1,16
1,2,16
6,3,9
3,4,12
4,5,11
7,6,5
5,7,9
10,8,1
9,9,1


# Create and Export Final Table

In [5]:
#Create age column and drop unneccesary columns

df['birth_date'] = pd.to_datetime(df['birth_date'])
opening_day = datetime.datetime(2024, 9, 5, 0, 0, 0) # September 5, 2024, 12:00 AM, opening day
df['age'] = ((opening_day - df['birth_date']) / pd.Timedelta(days=365)).astype(int)
df['gm_log_rtrvd'] = 0
df = df[['season', 'team', 'position', 'full_name', 'height', 'weight', 'age', 'years_exp', 'pfr_id', 'gm_log_rtrvd']].reset_index(drop=True)
display(df)

,season,team,position,full_name,height,weight,age,years_exp,pfr_id,gm_log_rtrvd
0,2024,PIT,RB,Cordarrelle Patterson,74.0,238,33,11,PattCo00,0
1,2024,MIA,RB,Raheem Mostert,70.0,197,32,9,MostRa00,0
2,2024,BAL,RB,Derrick Henry,75.0,247,30,8,HenrDe00,0
3,2024,MIN,RB,Aaron Jones,69.0,208,29,7,JoneAa00,0
4,2024,KC,RB,Samaje Perine,70.0,240,28,7,PeriSa00,0
...,...,...,...,...,...,...,...,...,...,...
95,2024,NE,RB,Terrell Jennings,72.0,225,23,0,JennTe00,0
96,2024,NYJ,RB,Braelon Allen,73.0,235,20,0,AlleBr05,0
97,2024,NYJ,RB,Isaiah Davis,73.0,220,22,0,DaviIs00,0
98,2024,MIA,RB,Jaylen Wright,71.0,210,21,0,WrigJa04,0


In [6]:
# Export final table

df.to_csv('../tables/players_xref.csv', index = False)